In [1]:
import obspy
from obspy.clients.fdsn.mass_downloader import CircularDomain, Restrictions, MassDownloader
import obspy
import numpy as np
import pandas as pd
from obspy import read
from obspy.core import Stream
import glob
import os
from scipy import signal
from datetime import datetime

ModuleNotFoundError: No module named 'obspy'

In [ ]:
# Program 1: P-Wave Early Detection
def download_early_pwave_data(event_time, latitude, longitude):
    domain = CircularDomain(
        latitude=latitude,
        longitude=longitude,
        minradius=0.0,
        maxradius=10.0 # 10 degrees (~1100 km)
    )
    restrictions = Restrictions(
        starttime=event_time - 30, # 30 seconds before
        endtime=event_time + 120,
        # 2 minutes after
        reject_channels_with_gaps=True,
        minimum_length=0.95,
        minimum_interstation_distance_in_m=5000, # 5 km minimum spacing
        # Prioritize high sample rate channels
        channel_priorities=["HH[ZNE]", "BH[ZNE]", "EH[ZNE]"],
        location_priorities=["", "00", "10"],
        # Additional parameters for data quality
        # minimum_sample_rate=100.0 # Ensure high sampling rate
    )
    mdl = MassDownloader()
    mdl.download(
        domain,
        restrictions,
        mseed_storage="waveforms_pwave",
        stationxml_storage="stations_pwave"
    )

In [ ]:
# Program 2: Ground Motion Prediction
def download_ground_motion_data(event_time, latitude, longitude):
    domain = CircularDomain(
        latitude=latitude,
        longitude=longitude,
        minradius=0.0,
        maxradius=20.0 # 20 degrees (~2200 km)
    )
    restrictions = Restrictions(
        starttime=event_time - 60, # 1 minute before
        endtime=event_time + 600, # 10 minutes after
        reject_channels_with_gaps=True,
        minimum_length=0.90,
        minimum_interstation_distance_in_m=20000, # 20 km minimum spacing
        # Include strong motion channels
        channel_priorities=["HN[ZNE]", "HH[ZNE]", "BH[ZNE]"],
        location_priorities=["", "00", "10"],
        # Less strict on sample rate for surface waves
        # minimum_sample_rate=50.0
    )
    mdl = MassDownloader()
    mdl.download(
        domain,
        restrictions,
        mseed_storage="waveforms_motion",
        stationxml_storage="stations_motion"
    )

In [ ]:
# Program 3: Aftershock Analysis
def download_aftershock_data(event_time, latitude, longitude):
  domain = CircularDomain(
    latitude=latitude,
    longitude=longitude,
    minradius=0.0,
    maxradius=15.0 # 15 degrees (~1650 km)
  )
  restrictions = Restrictions(
    starttime=event_time, # Start at mainshock
    endtime=event_time + 30 * 86400,
    # 30 days after
    reject_channels_with_gaps=False,
    # Allow some gaps due to long duration
    minimum_length=0.70,
    # More lenient minimum length
    minimum_interstation_distance_in_m=50000, # 50 km minimum spacing
    # Include broader range of channels
    channel_priorities=["HH[ZNE]", "BH[ZNE]", "HN[ZNE]", "EH[ZNE]"],
    location_priorities=["", "00", "10"],
    # Less strict on sample rate
    minimum_sample_rate=20.0
  )
  mdl = MassDownloader()
  mdl.download(
    domain,
    restrictions,
    mseed_storage="waveforms_aftershock",
    stationxml_storage="stations_aftershock"
  )# Example event (2011 Tohoku earthquake)
event_time = obspy.UTCDateTime(2011, 3, 11, 5, 47, 32)
event_latitude = 37.52
event_longitude = 143.04
# Download data for each analysis type
download_ground_motion_data(event_time, event_latitude, event_longitude)

In [ ]:
# Example event (2011 Tohoku earthquake)
event_time = obspy.UTCDateTime(2011, 3, 11, 5, 47, 32)
event_latitude = 37.52
event_longitude = 143.04
# Download data for each analysis type
download_ground_motion_data(event_time, event_latitude, event_longitude)

[2025-02-15 15:30:56,246] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for AUSPASS, BGR, EIDA, EMSC, ETH, GEOFON, GEONET, GFZ, ICGC, IESDMC, INGV, IPGP, ISC, KNMI, KOERI, LMU, NCEDC, NIEP, NOA, RESIF, RESIFPH5, SCEDC, TEXNET, UIB-NORSAR, USGS, USP, ORFEUS, IRIS.
[2025-02-15 15:30:56,849] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'ISC' as it does not have 'dataselect' and/or 'station' services.
[2025-02-15 15:30:56,895] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'USGS' as it does not have 'dataselect' and/or 'station' services.
[2025-02-15 15:30:57,372] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'EMSC' as it does not have 'dataselect' and/or 'station' services.
[2025-02-15 15:30:57,591] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'RESIFPH5' as it does not have 'dataselect' and/or 'station' services.
[2025-02-15 15:30:57,894] - obspy.clients.fdsn.mass_downloader - WARNING: Failed t

In [ ]:
def process_station_metadata(station_dir):
    """
    Process station XML files to extract station metadata
    """
    metadata = []
    for xml_file in glob.glob(os.path.join(station_dir, "*.xml")):
        inventory = obspy.read_inventory(xml_file)
        for network in inventory:
            for station in network:
                station_info = {
                    'station_id': f"{network.code}.{station.code}",
                    'latitude': station.latitude,
                    'longitude': station.longitude,
                    'elevation': station.elevation,
                    'distance_to_event': None  # Will be calculated later
                }
                metadata.append(station_info)
    return pd.DataFrame(metadata)

def process_waveforms(waveform_dir, sampling_rate=100.0, window_size=60):
    """
    Process MSEED waveform files into fixed-length windows
    """
    features = []
    for mseed_file in glob.glob(os.path.join(waveform_dir, "*.mseed")):
        try:
            # Read the stream
            st = read(mseed_file)

            # Process each trace in the stream
            for tr in st:
                # Preprocess the trace
                tr = preprocess_trace(tr, sampling_rate)

                # Extract windows
                windows = extract_windows(tr, window_size, sampling_rate)

                for window in windows:
                    feature_dict = extract_features(window)
                    feature_dict.update({
                        'station_id': f"{tr.stats.network}.{tr.stats.station}",
                        'channel': tr.stats.channel,
                        'start_time': tr.stats.starttime
                    })
                    features.append(feature_dict)

        except Exception as e:
            print(f"Error processing {mseed_file}: {str(e)}")
            continue

    return pd.DataFrame(features)

def preprocess_trace(trace, sampling_rate):
    """
    Preprocess a single seismic trace
    """
    trace = trace.copy()

    # Remove mean and trend
    trace.detrend('demean')
    trace.detrend('linear')

    # Resample if necessary
    if trace.stats.sampling_rate != sampling_rate:
        trace.resample(sampling_rate)

    # Taper edges
    trace.taper(max_percentage=0.05, type='hann')

    return trace

def extract_windows(trace, window_size, sampling_rate):
    """
    Extract fixed-length windows from a trace
    """
    # Calculate number of samples per window
    samples_per_window = int(window_size * sampling_rate)

    # Get data array
    data = trace.data

    # Split into windows
    windows = []
    for i in range(0, len(data) - samples_per_window, samples_per_window):
        window = data[i:i + samples_per_window]
        if len(window) == samples_per_window:
            windows.append(window)

    return windows

def extract_features(window):
    """
    Extract features from a window of seismic data
    """
    features = {
        # Time domain features
        'mean': np.mean(window),
        'std': np.std(window),
        'max': np.max(np.abs(window)),
        'peak_to_peak': np.ptp(window),
        'rms': np.sqrt(np.mean(np.square(window))),

        # Frequency domain features
        'dominant_freq': compute_dominant_frequency(window),
        'spectral_centroid': compute_spectral_centroid(window),

        # Energy features
        'energy': np.sum(np.square(window)),

        # Store raw waveform for ML model
        'waveform': window.tolist()
    }
    return features

def compute_dominant_frequency(window, fs=100.0):
    """
    Compute the dominant frequency in a window
    """
    freqs, psd = signal.welch(window, fs=fs)
    return freqs[np.argmax(psd)]

def compute_spectral_centroid(window, fs=100.0):
    """
    Compute the spectral centroid of a window
    """
    freqs, psd = signal.welch(window, fs=fs)
    return np.sum(freqs * psd) / np.sum(psd)

def create_ml_dataset(waveform_dir, station_dir, event_coords):
    """
    Create complete dataset for ML modeling
    """
    # Process station metadata
    station_df = process_station_metadata(station_dir)

    # Calculate distances to event
    for idx, row in station_df.iterrows():
        station_df.loc[idx, 'distance_to_event'] = obspy.geodetics.locations2degrees(
            event_coords['latitude'],
            event_coords['longitude'],
            row['latitude'],
            row['longitude']
        )

    # Process waveforms and extract features
    features_df = process_waveforms(waveform_dir)

    # Merge station metadata with features
    final_df = features_df.merge(station_df, on='station_id')

    return final_df

# Example usage
event_coords = {
    'latitude': 37.52,
    'longitude': 143.04
}

dataset = create_ml_dataset(
    waveform_dir="waveforms_motion",
    station_dir="stations_motion",
    event_coords=event_coords
)

<ipython-input-8-978420cc01b4>:136: RuntimeWarning: invalid value encountered in scalar divide
  return np.sum(freqs * psd) / np.sum(psd)


In [ ]:
dataset.to_csv('dataset.csv', index=False)
dataset = pd.read_csv('dataset.csv')

def clean_seismic_dataset(df):
    """
    Clean and prepare seismic dataset for ML modeling
    """
    # Create a copy to avoid modifying original data
    df_clean = df.copy()

    # Convert mean to float64
    df_clean['mean'] = pd.to_numeric(df_clean['mean'], errors='coerce')

    # Convert start_time to datetime
    df_clean['start_time'] = pd.to_datetime(df_clean['start_time'])

    # Convert waveform from string/object to numpy arrays
    df_clean['waveform'] = df_clean['waveform'].apply(lambda x: np.array(x) if isinstance(x, list) else x)

    # Drop any rows with missing values
    df_clean = df_clean.dropna()

    # Create feature matrix X and target y (example using PGA prediction)
    feature_columns = [
        'mean', 'std', 'max', 'peak_to_peak', 'rms',
        'dominant_freq', 'spectral_centroid', 'energy',
        'latitude', 'longitude', 'elevation', 'distance_to_event'
    ]

    # Encode categorical variables
    df_clean['channel_encoded'] = pd.Categorical(df_clean['channel']).codes
    feature_columns.append('channel_encoded')

    # Create time-based features
    df_clean['hour'] = df_clean['start_time'].dt.hour
    df_clean['minute'] = df_clean['start_time'].dt.minute
    feature_columns.extend(['hour', 'minute'])

    # Scale numerical features
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    df_clean[feature_columns] = scaler.fit_transform(df_clean[feature_columns])

    # Create feature matrix
    X = df_clean[feature_columns]

    # For demonstration, let's use 'max' as target (you can modify based on your needs)
    y = df_clean['max']

    # Create metadata dictionary for reference
    metadata = {
        'feature_columns': feature_columns,
        'scaler': scaler,
        'station_ids': df_clean['station_id'].unique(),
        'channels': df_clean['channel'].unique(),
        'time_range': (df_clean['start_time'].min(), df_clean['start_time'].max())
    }

    return X, y, metadata

def split_seismic_dataset(X, y, test_size=0.2, random_state=42):
    """
    Split dataset while ensuring no station data leakage
    """
    from sklearn.model_selection import train_test_split

    # Split while maintaining temporal order
    train_idx = int(len(X) * (1 - test_size))

    X_train = X.iloc[:train_idx]
    X_test = X.iloc[train_idx:]
    y_train = y.iloc[:train_idx]
    y_test = y.iloc[train_idx:]

    return X_train, X_test, y_train, y_test

# Example usage
# Assuming your DataFrame is called 'dataset'
X, y, metadata = clean_seismic_dataset(dataset)
X_train, X_test, y_train, y_test = split_seismic_dataset(X, y)

print("Dataset shapes:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print("\nFeatures used:", metadata['feature_columns'])
dataset

Dataset shapes:
X_train: (557, 15)
X_test: (140, 15)

Features used: ['mean', 'std', 'max', 'peak_to_peak', 'rms', 'dominant_freq', 'spectral_centroid', 'energy', 'latitude', 'longitude', 'elevation', 'distance_to_event', 'channel_encoded', 'hour', 'minute']


,mean,std,max,peak_to_peak,rms,dominant_freq,spectral_centroid,energy,waveform,station_id,channel,start_time,latitude,longitude,elevation,distance_to_event
0,199.88429148806142,2.621904e+02,5.679320e+02,1.015781e+03,3.296931e+02,0.390625,1.272811,6.521852e+08,"[-0.0, -0.009376589721739397, -0.0068151650654...",YP.NE49,BHE,2011-03-11T05:46:32.000000Z,43.393002,128.753006,561.0,12.332148
1,-277.8450263388052,2.280927e+02,6.283838e+02,8.543353e+02,3.594776e+02,0.390625,1.733822,7.753449e+08,"[-371.5836946788273, -377.89290694657325, -384...",YP.NE49,BHE,2011-03-11T05:46:32.000000Z,43.393002,128.753006,561.0,12.332148
2,8451.704919313446,6.019199e+04,3.755226e+05,4.808498e+05,6.078245e+04,0.390625,0.407042,2.216704e+13,"[-310.9204698996203, -302.87394285532713, -296...",YP.NE49,BHE,2011-03-11T05:46:32.000000Z,43.393002,128.753006,561.0,12.332148
3,256404.0516045682,9.018107e+05,1.757649e+06,3.095176e+06,9.375529e+05,0.390625,0.400405,5.274033e+15,"[375891.7614973526, 376325.0706626536, 376818....",YP.NE49,BHE,2011-03-11T05:46:32.000000Z,43.393002,128.753006,561.0,12.332148
4,-250741.61130331422,1.931103e+06,3.690223e+06,6.840455e+06,1.947314e+06,0.390625,0.408584,2.275218e+16,"[-114843.71144881008, -116380.5778463769, -118...",YP.NE49,BHE,2011-03-11T05:46:32.000000Z,43.393002,128.753006,561.0,12.332148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,-15288.615791896025,2.610845e+06,6.104928e+06,1.026215e+07,2.610890e+06,0.390625,0.404182,4.090047e+16,"[4157227.0220557377, 4152081.675935657, 414688...",YP.NE59,BHZ,2011-03-11T05:46:32.000001Z,44.095901,125.961098,214.0,14.464702
694,6116.504804129967,2.880794e+03,8.882021e+03,8.882122e+03,6.760962e+03,0.390625,0.502543,2.742636e+11,"[-0.0, -0.0409217828917219, -0.095917592359669...",IC.MDJ,BHN,2011-03-11T05:46:32.023139Z,44.617000,129.590800,270.0,12.350914
695,5714.9810437060405,9.094264e+02,7.354029e+03,3.256357e+03,5.786887e+03,0.390625,0.850893,2.009284e+11,"[7246.196999591337, 7244.252495888363, 7242.74...",IC.MDJ,BHN,2011-03-11T05:46:32.023139Z,44.617000,129.590800,270.0,12.350914
696,1670.7016407740532,3.629282e+04,1.802792e+05,2.661009e+05,3.633125e+04,0.390625,0.406652,7.919759e+12,"[4089.1415900902675, 4079.2429558037447, 4068....",IC.MDJ,BHN,2011-03-11T05:46:32.023139Z,44.617000,129.590800,270.0,12.350914
